In [1]:
!pip install -q --upgrade bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.7 MB/s eta 0:00:00


In [2]:
!pip install -U -q bitsandbytes

In [3]:
import os
from openai import OpenAI
from google.colab import userdata
from google.colab import drive
import gc

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
from huggingface_hub import login
from IPython.display import Markdown, display, update_display

In [5]:
import gradio as gr
from IPython.display import Markdown, display

In [6]:
!pip install textblob

In [7]:
from google.cloud import translate_v2
from textblob import TextBlob

In [8]:
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 91.3 MB/s eta 0:00:00


In [9]:
import pycountry

In [10]:
!pip install -q pytube

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.3 MB/s eta 0:00:00


In [11]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 65.6 MB/s eta 0:00:00


In [12]:
!pip install -q faster-whisper yt-dlp pydub torch
!pip install -q silero-vad

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 51.8 MB/s eta 0:00:00


In [13]:
import yt_dlp
from pydub import AudioSegment
from faster_whisper import WhisperModel
import torch
import numpy as np
from silero_vad import load_silero_vad, get_speech_timestamps, read_audio, VADIterator
import uuid

In [15]:
hf_token = userdata.get('HF_TOKEN_1')
login(hf_token, add_to_git_credential = True)

In [16]:
LLAMA = "meta-llama/Llama-3.2-3B-Instruct"
QWEN = "Qwen/Qwen3-4B-Instruct-2507"
PHI = "microsoft/Phi-4-mini-instruct"
DEEPSEEK = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
Gemma = 'google/gemma-3-4b-it'

In [17]:
%%writefile cookies.txt
# Netscape HTTP Cookie File
# http://curl.haxx.se/rfc/cookie_spec.html
# This file was generated by Cookie-Editor
#HttpOnly_.youtube.com	TRUE	/	TRUE	1797881850	__Secure-3PSID	g.a0003ghAhLymsxWCquG0lizRx5WxvXcI1nQVc1CAOMM4dj6aoALOonWHxv4N253gV9vEq6xH5QACgYKAZASARISFQHGX2Mi_pN_g7OwFplx3yV4KI4_LxoVAUF8yKpv8LZKHs2gkeben9s_6bX40076
#HttpOnly_.youtube.com	TRUE	/	TRUE	1794862784	__Secure-1PSIDTS	sidts-CjQBwQ9iI7x6YTLun349txwhf7_lSVGqqjAv64-sw6txLIE-U_eNCZWmwVi3Nw10YYJ12tivEAA
.youtube.com	TRUE	/	TRUE	1797881850	SAPISID	XSVT7wZzdRdP78oF/AGdL_7xEKaCrXnOYh
#HttpOnly_.youtube.com	TRUE	/	TRUE	1794862784	__Secure-1PSIDCC	AKEyXzWT8TJuKk7SPjUA9OpmJmuWCpfes_lz59pTS4vNiiHwHIAM2itsOiB6RIGf3afGlLFHoPg
#HttpOnly_.youtube.com	TRUE	/	TRUE	1797881850	SSID	AbjVZy1Ds6pIZYX81
.youtube.com	TRUE	/	TRUE	1797881850	__Secure-1PAPISID	XSVT7wZzdRdP78oF/AGdL_7xEKaCrXnOYh
#HttpOnly_.youtube.com	TRUE	/	TRUE	1797881850	__Secure-1PSID	g.a0003ghAhLymsxWCquG0lizRx5WxvXcI1nQVc1CAOMM4dj6aoALO_eq88Vjar1rU9qwuEyro8gACgYKAQkSARISFQHGX2MibtucPY2oXt4tn2aRMUNBPxoVAUF8yKoW5_tefYXw2fpBR8w6RnrV0076
.youtube.com	TRUE	/	TRUE	1797881850	__Secure-3PAPISID	XSVT7wZzdRdP78oF/AGdL_7xEKaCrXnOYh
#HttpOnly_.youtube.com	TRUE	/	TRUE	1794862784	__Secure-3PSIDCC	AKEyXzXDKnfvTjpWzifDYE3BWcRZGbXNQ1S3HwJPsVZx-Palwzxwv7BZUpfSygDZ1m4zrmBcjBU
#HttpOnly_.youtube.com	TRUE	/	TRUE	1794862784	__Secure-3PSIDTS	sidts-CjQBwQ9iI7x6YTLun349txwhf7_lSVGqqjAv64-sw6txLIE-U_eNCZWmwVi3Nw10YYJ12tivEAA
#HttpOnly_.youtube.com	TRUE	/	TRUE	1795717524	LOGIN_INFO	AFmmF2swRAIgDJxtYbVJgdyjA4grs4VqOhUOrOX0SNiSiJNDLU2Xj24CIDOyUV8oEidkYvPex2zj-oqf_RXnyjGsmwKzf4GBmmlC:QUQ3MjNmd2kzY1RZMng1ODFhT0FsQTM2ZFlRbmNwYnVBbWxnNFhCcEZJcDFkS2JiQ0d2VHJjc3J0akExRkpWQngtT1VhVE42bW1ZclZpUnJwUmpmbWJsR050ejVyc2ltbEFOUGpnRG90b0tOUDBrQzMxdnVjSy0xdnZseGNPR3lDODU4OXhRUW9DZm02VDZGSUxKeXB4OE9PY2tCV1c4YW13
.youtube.com	TRUE	/	TRUE	1797886680	PREF	f4=4000000&f6=40000000&tz=Asia.Calcutta&f5=20000&f7=18100

Writing cookies.txt


In [18]:
def _download_if_youtube(source):
    if "youtube.com" in source or "youtu.be" in source:

        # Generate unique ID to avoid overwriting
        unique = str(uuid.uuid4())[:8]
        filename = f"audio_{unique}.%(ext)s"

        ydl_opts = {
            "format": "bestaudio/best",
            "outtmpl": filename,
            "quiet": True,
            "extractor_args": {"youtube": {"player_client": ["default"]}},
            "cookiefile": "cookies.txt",
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(source, download=True)
            return ydl.prepare_filename(info)
    else:
        return source


def _to_wav(path):
    unique = str(uuid.uuid4())[:8]
    wav_path = f"audio_{unique}.wav"
    AudioSegment.from_file(path).export(wav_path, format="wav")
    return wav_path

def transcription_whisper(source):
    torch.cuda.empty_cache()
    gc.collect()

    device = "cuda" if torch.cuda.is_available() else "cpu"
    compute = "float16" if device == "cuda" else "int8"

    model = WhisperModel('medium', device=device, compute_type=compute)

    file_path = _download_if_youtube(source)

    wav_path = _to_wav(file_path)

    segments, info = model.transcribe(wav_path)

    result = []
    formatted_output = "**TRANSCRIPTION**\n" + "="*50 + "\n\n"

    for seg in segments:
        result.append({
            "start": seg.start,
            "end": seg.end,
            "text": seg.text.strip()
        })
        formatted_output += f"[{seg.start:.2f}s - {seg.end:.2f}s]\n{seg.text.strip()}\n\n"

    del model
    gc.collect()
    torch.cuda.empty_cache()

    return formatted_output, result

In [19]:
with open("cookies.txt", "r") as f:
    data = f.read()

print(data)


# Netscape HTTP Cookie File
# http://curl.haxx.se/rfc/cookie_spec.html
# This file was generated by Cookie-Editor
#HttpOnly_.youtube.com	TRUE	/	TRUE	1797881850	__Secure-3PSID	g.a0003ghAhLymsxWCquG0lizRx5WxvXcI1nQVc1CAOMM4dj6aoALOonWHxv4N253gV9vEq6xH5QACgYKAZASARISFQHGX2Mi_pN_g7OwFplx3yV4KI4_LxoVAUF8yKpv8LZKHs2gkeben9s_6bX40076
#HttpOnly_.youtube.com	TRUE	/	TRUE	1794862784	__Secure-1PSIDTS	sidts-CjQBwQ9iI7x6YTLun349txwhf7_lSVGqqjAv64-sw6txLIE-U_eNCZWmwVi3Nw10YYJ12tivEAA
.youtube.com	TRUE	/	TRUE	1797881850	SAPISID	XSVT7wZzdRdP78oF/AGdL_7xEKaCrXnOYh
#HttpOnly_.youtube.com	TRUE	/	TRUE	1794862784	__Secure-1PSIDCC	AKEyXzWT8TJuKk7SPjUA9OpmJmuWCpfes_lz59pTS4vNiiHwHIAM2itsOiB6RIGf3afGlLFHoPg
#HttpOnly_.youtube.com	TRUE	/	TRUE	1797881850	SSID	AbjVZy1Ds6pIZYX81
.youtube.com	TRUE	/	TRUE	1797881850	__Secure-1PAPISID	XSVT7wZzdRdP78oF/AGdL_7xEKaCrXnOYh
#HttpOnly_.youtube.com	TRUE	/	TRUE	1797881850	__Secure-1PSID	g.a0003ghAhLymsxWCquG0lizRx5WxvXcI1nQVc1CAOMM4dj6aoALO_eq88Vjar1rU9qwuEyro8gACgYKAQkSARIS

In [ ]:
transcription_whisper('https://www.youtube.com/watch?v=-9eA4IgS2oQ')


--- Transcription Output ---

[0.00 → 14.00] みなさんこんにちは。私は今、何駅に着いたでしょうか。駅の名前を聞き取ってください。

[14.00 → 34.00] どうでしたか。知らないと聞き取れなかったかもしれません。代々木公園と言っていました。

[34.00 → 40.00] 今日は東京の渋谷にある代々木公園を紹介します。

[41.00 → 49.00] まずは改札を出て、4番出口へ向かいます。

[53.00 → 58.00] 階段を上って、外に出ます。

[62.00 → 68.00] 右の方向に3分ぐらい歩いていくと公園の門があります。

[71.00 → 75.00] ここから入りましょう。西門です。

[78.00 → 80.00] 地図を見てみましょう。

[81.00 → 94.00] この現在地。現在はpresent。地はplace。つまり今いる場所のことです。広いですね。

[95.00 → 100.00] 代々木公園は東京で6番目に広い公園だそうです。

[101.00 → 106.00] どこに行っていいかわからないので、とりあえずあちらに行きましょう。

[107.00 → 112.00] 走っている人がいますね。と思ったら歩いていますね。

[113.00 → 116.00] 坂なんで疲れたんですかね。

[119.00 → 122.00] 立ち入りをご遠慮くださいと書いてあります。

[125.00 → 128.00] 入らないでくださいという意味ですね。

[133.00 → 135.00] ちょっとこっちへ行ってみましょう。

[140.00 → 142.00] 落ち葉がたくさんありますね。

[144.00 → 153.00] この日は11月26日でした。なので公園を見るには、11月中旬頃がいいかもしれないですね。

[154.00 → 163.00] 何かありますよ。行ってみましょう。

[164.00 → 168.00] 日本初飛行の地と書いてあります。

[169.00 → 177.00] ここで初めて飛行機が飛んだんですね。ここは昔陸軍の基地だったそうです。

[178.00 → 181.00] 知らなかったです。

[182.00 → 188.00] この日は朝まで雨が

[{'start': 0.0,
  'end': 14.0,
  'text': 'みなさんこんにちは。私は今、何駅に着いたでしょうか。駅の名前を聞き取ってください。'},
 {'start': 14.0,
  'end': 34.0,
  'text': 'どうでしたか。知らないと聞き取れなかったかもしれません。代々木公園と言っていました。'},
 {'start': 34.0, 'end': 40.0, 'text': '今日は東京の渋谷にある代々木公園を紹介します。'},
 {'start': 41.0, 'end': 49.0, 'text': 'まずは改札を出て、4番出口へ向かいます。'},
 {'start': 53.0, 'end': 58.0, 'text': '階段を上って、外に出ます。'},
 {'start': 62.0, 'end': 68.0, 'text': '右の方向に3分ぐらい歩いていくと公園の門があります。'},
 {'start': 71.0, 'end': 75.0, 'text': 'ここから入りましょう。西門です。'},
 {'start': 78.0, 'end': 80.0, 'text': '地図を見てみましょう。'},
 {'start': 81.0,
  'end': 94.0,
  'text': 'この現在地。現在はpresent。地はplace。つまり今いる場所のことです。広いですね。'},
 {'start': 95.0, 'end': 100.0, 'text': '代々木公園は東京で6番目に広い公園だそうです。'},
 {'start': 101.0, 'end': 106.0, 'text': 'どこに行っていいかわからないので、とりあえずあちらに行きましょう。'},
 {'start': 107.0, 'end': 112.0, 'text': '走っている人がいますね。と思ったら歩いていますね。'},
 {'start': 113.0, 'end': 116.0, 'text': '坂なんで疲れたんですかね。'},
 {'start': 119.0, 'end': 122.0, 'text': '立ち入りをご遠慮くださいと書いてあります。'},
 {'start': 125.0, 'end': 1

In [20]:
system_prompt = """
You are an expert assistant that generates clear, concise, and well-structured
Minutes of Meeting (MOM) documents from raw meeting transcripts.

Your output must be in clean Markdown format (without code blocks) and must include:

- **Meeting Summary:** A brief overview of the meeting context, agenda, and participants (if mentioned).
- **Key Discussion Points:** Major topics, decisions, or debates.
- **Takeaways:** Important insights, conclusions, and agreements.
- **Action Items:** Actionable tasks with responsible owners and deadlines
  (e.g., "John will prepare the project plan by Monday").

Guidelines:
- Write in professional, easy-to-read language.
- Summarize meaningfully; avoid filler words or irrelevant content.
- Omit transcription artifacts (e.g., “um”, “okay”, “yeah”).
- Do not include timestamps.
- Maintain a formal and factual tone while being concise.
- Focus entirely on clarity, structure, and readability.
"""


In [21]:
def user_prompt_for(source):
    formatted_output, segments = transcription_whisper(source)

    transcript_text = " ".join(seg["text"] for seg in segments)

    user_prompt = f"""
Please write well-structured **Minutes of Meeting (MOM)** in Markdown format (without code blocks), including:

- **Summary:** Include attendees, location, and date if mentioned.
- **Key Discussion Points:** List the main topics or discussions.
- **Takeaways:** Summaries of conclusions or insights.
- **Action Items:** Tasks with clear owners and deadlines.

Transcription:
{transcript_text}
"""
    return user_prompt

In [22]:
def messages_for(source):

  messages = [
    {'role': 'system', 'content': system_prompt},
    {'role': 'user', 'content': user_prompt_for(source)}
    ]

  return messages

In [23]:
quant_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_quant_type = 'nf4'
)

In [24]:
def generate(model_name, source):
  messages = messages_for(source)
  tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(messages,return_tensors = 'pt',add_generation_prompt = True).to('cuda')
  model = AutoModelForCausalLM.from_pretrained(model_name, device_map = 'auto',quantization_config = quant_config)
  streamer = TextStreamer(tokenizer)
  outputs = model.generate(inputs, streamer = streamer , max_new_tokens = 5000)
  result = tokenizer.decode(outputs[0],skip_special_tokens=True)

  mom_output = result

  if '<|start_header_id|>assistant<|end_header_id|>' in mom_output:
    mom_output = mom_output.split('<|start_header_id|>assistant<|end_header_id|>')[-1]
  elif 'assistant' in mom_output:
    parts = mom_output.split('assistant')
    if len(parts) > 1:
      mom_output = parts[-1]

  mom_output = mom_output.replace('<|eot_id|>', '').replace('<|end_header_id|>', '').strip()

  if '**Minutes of Meeting' in mom_output:
    mom_output = mom_output.split('**Minutes of Meeting')[1]
    mom_output = '**Minutes of Meeting' + mom_output
  elif '**MINUTES' in mom_output:
    mom_output = mom_output.split('**MINUTES')[1]
    mom_output = '**MINUTES' + mom_output

  del model, inputs, tokenizer, outputs
  gc.collect()
  torch.cuda.empty_cache()

  yield mom_output.strip()

In [25]:
def valid_language(lang):
    return bool (
        pycountry.languages.get(name = lang.capitalize()) or
        pycountry.languages.get(alpha_2 = lang.lower()) or
        pycountry.languages.get(alpha_3 = lang.lower())
    )

In [26]:
system_prompt_translate = "You are a translation assistant. Given a target language and some content, translate the content accurately into that language, preserving meaning, tone, and style, and return only the translated text. Also maintain proper format."

In [27]:
def user_prompt_translate(source, lang):
    if not valid_language(lang):
        return f"Invalid language: {lang}. Please provide a valid language name or code."

    transcript_text, _ = transcription_whisper(source)

    lines = transcript_text.split('\n')
    text_lines = []
    for line in lines:
        if line.startswith('**') or line.startswith('=') or line.startswith('[') or not line.strip():
            continue
        text_lines.append(line.strip())

    transcript_text = " ".join(text_lines)

    max_chars = 3000
    if len(transcript_text) > max_chars:
        transcript_text = transcript_text[:max_chars] + "..."

    user_prompt = f"""Translate the following text into {lang}.

Instructions:
- Provide ONLY the translation in {lang}
- Do NOT add any explanations or comments
- Preserve the original meaning and tone
- Keep formatting simple and clean

Text to translate:
{transcript_text}

{lang} translation:"""

    return user_prompt


In [28]:
def messages_for_translate(source,lang):
  messages = [
      {'role':'system','content':system_prompt_translate},
      {'role':'user','content':user_prompt_translate(source,lang)}
  ]
  return messages

In [35]:
def translate_transcribe(model_name, source, lang):
  messages = messages_for_translate(source, lang)
  tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(messages,return_tensors = 'pt',add_generation_prompt = True).to('cuda')
  model = AutoModelForCausalLM.from_pretrained(model_name, device_map = 'auto',quantization_config = quant_config)
  streamer = TextStreamer(tokenizer)
  outputs = model.generate(inputs, streamer = streamer , max_new_tokens = 5000)
  result = tokenizer.decode(outputs[0],skip_special_tokens=True)

  translate_output = result

  if '<|start_header_id|>assistant<|end_header_id|>' in translate_output:
    translate_output = translate_output.split('<|start_header_id|>assistant<|end_header_id|>')[-1]
  elif 'assistant' in translate_output:
    parts = translate_output.split('assistant')
    if len(parts) > 1:
      translate_output = parts[-1]

  translate_output = translate_output.replace('<|eot_id|>', '').replace('<|end_header_id|>', '').strip()

  if 'translation:' in translate_output.lower():
    translate_output = translate_output.split('translation:')[-1].strip()

  if "Here's an edited version:" in translate_output:
    translate_output = translate_output.split("Here's an edited version:")[0].strip()

  translate_output = translate_output.replace('assistant', '').strip()

  sentences = translate_output.split('. ')
  paragraphs = []
  current_para = []
  sentence_count = 0

  for sentence in sentences:
    current_para.append(sentence.strip())
    sentence_count += 1

    if sentence_count >= 4:
      paragraphs.append('. '.join(current_para) + '.')
      current_para = []
      sentence_count = 0

  if current_para:
    paragraphs.append('. '.join(current_para) + ('.' if not current_para[-1].endswith('.') else ''))


  formatted_output = '\n\n'.join(paragraphs)

  del model, inputs, tokenizer, outputs
  gc.collect()
  torch.cuda.empty_cache()

  yield formatted_output

In [36]:
def translate_transcribe_gemma(Gemma, source, lang):
  messages = [{'role':'user','content':user_prompt_translate(source,lang)}]
  tokenizer = AutoTokenizer.from_pretrained(Gemma,trust_remote_code=True)
  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(messages,return_tensors = 'pt',add_generation_prompt = True).to('cuda')
  model = AutoModelForCausalLM.from_pretrained(Gemma, device_map = 'auto',quantization_config = quant_config)
  streamer = TextStreamer(tokenizer)
  outputs = model.generate(inputs, streamer = streamer , max_new_tokens = 5000)
  result = tokenizer.decode(outputs[0],skip_special_tokens=True)

  translate_output = result

  if '<|start_header_id|>assistant<|end_header_id|>' in translate_output:
    translate_output = translate_output.split('<|start_header_id|>assistant<|end_header_id|>')[-1]
  elif 'assistant' in translate_output:
    parts = translate_output.split('assistant')
    if len(parts) > 1:
      translate_output = parts[-1]

  translate_output = translate_output.replace('<|eot_id|>', '').replace('<|end_header_id|>', '').strip()

  if 'translation:' in translate_output.lower():
    translate_output = translate_output.split('translation:')[-1].strip()

  if "Here's an edited version:" in translate_output:
    translate_output = translate_output.split("Here's an edited version:")[0].strip()

  translate_output = translate_output.replace('assistant', '').strip()

  del model, inputs, tokenizer, outputs
  gc.collect()
  torch.cuda.empty_cache()

  yield translate_output

In [37]:
def optimize(model_name, source):
  if model_name == 'LLAMA':
    result = generate(LLAMA, source)

  elif model_name == 'PHI':
    result = generate(PHI, source)

  elif model_name == 'QWEN':
    result = generate(QWEN, source)

  elif model_name == 'DEEPSEEK':
    result = generate(DEEPSEEK, source)

  for chunk in result:
    yield chunk

In [38]:
def optimize_translate(model_name, source, lang):
  if model_name == 'LLAMA':
    translate= translate_transcribe(LLAMA, source, lang)
  elif model_name == 'PHI':
    translate= translate_transcribe(PHI, source, lang)
  elif model_name == 'QWEN':
    translate= translate_transcribe(QWEN, source, lang)
  elif model_name == 'DEEPSEEK':
    translate= translate_transcribe(DEEPSEEK, source, lang)
  elif model_name == 'Gemma':
    translate= translate_transcribe_gemma(Gemma, source, lang)
  for chunk_tr in translate:
    yield chunk_tr

In [39]:
css = """
#file-box {
    min-height: 500px !important;
}
#file-box button {
    height: 100% !important;
    width: 100% !important;
    display: flex !important;
    flex-direction: column !important;
    align-items: center !important;
    justify-content: center !important;
    margin: 0 !important;
    padding: 0 !important;
}
#box {
  min-height: 550px !important;
}
"""

In [ ]:
import gradio as gr

with gr.Blocks(css = css) as ui:
    gr.Markdown("## Transcription & MOM Generator & Translator")

    with gr.Row():


        with gr.Column(scale=2):
            input_file = gr.File(label="Upload Audio/Video ", file_types=["audio", "video"], elem_id="file-box")
            input_link = gr.Textbox(
                label="YouTube Link (optional)",
                lines=2
            )


        with gr.Column(scale=2):
            output_transcription = gr.Textbox(label="Transcription", lines=25, elem_id = 'box')
            transcribe = gr.Button("Transcribe", variant="primary", scale=2)


        with gr.Column(scale=2):
            output_summary = gr.Textbox(label="MOM Output", lines = 25, elem_id = 'box')
            summarize = gr.Button("Summarize", variant="secondary", scale=2)

        with gr.Column(scale = 2):
            output_translate = gr.Textbox(label = 'Translation Ouptut', lines = 20)
            language_input = gr.Textbox(label="Target Language", value="English", lines=1)
            translate = gr.Button('Translate', scale=2)

    with gr.Row():
      model = gr.Dropdown(
                ["LLAMA", "PHI", "QWEN", "DEEPSEEK",'Gemma'],
                label="Choose Your Model",
                value="LLAMA"
            )

    transcribe.click(
        fn=lambda source: transcription_whisper(source)[0],
        inputs=[input_link],
        outputs=[output_transcription]
    )

    summarize.click(
        fn=optimize,
        inputs=[model, input_link],
        outputs=[output_summary]
    )

    translate.click(
        fn = optimize_translate ,inputs = [model, input_link, language_input],
        outputs = [output_translate ]
    )

ui.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7d662c9892966ada77.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 16 Nov 2025

You are a translation assistant. Given a target language and some content, translate the content accurately into that language, preserving meaning, tone, and style, and return only the translated text. Also maintain proper format.<|eot_id|><|start_header_id|>user<|end_header_id|>

Translate the following text into English.

Instructions:
- Provide ONLY the translation in English
- Do NOT add any explanations or comments
- Preserve the original meaning and tone
- Keep formatting simple and clean

Text to translate:
みなさんこんにちは。私は今何駅に着いたでしょうか。 駅の名前を聞き取ってください。 どうでしたか? 知らないと聞き取れなかったかもしれません。 代々木公園と言っていました。 今日は東京の渋谷にある代々木公園を紹介します。 まずは改札を出て、4番出口へ向かいます。 階段を登って、外に出ます。 右の方向に3分ぐらい歩いていくと公園の門があります。 ここから入りましょう。西門です。 地図を見てみましょう。 この現在地。現在はpresent。 地はplace、つまり今いる場所のことです。 広いですね。 代々木公園は東京で6番目に広い公園だそうです。 どこに行っていいかわからないので、とりあえずあちらに行きましょう。 走っている人がいますね。 と思ったら歩いていますね。 坂なんで疲れたんですかね。 立ち入りをご遠

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

<|im_start|>system
You are a translation assistant. Given a target language and some content, translate the content accurately into that language, preserving meaning, tone, and style, and return only the translated text. Also maintain proper format.<|im_end|>
<|im_start|>user
Translate the following text into English.

Instructions:
- Provide ONLY the translation in English
- Do NOT add any explanations or comments
- Preserve the original meaning and tone
- Keep formatting simple and clean

Text to translate:
みなさんこんにちは。私は今何駅に着いたでしょうか。 駅の名前を聞き取ってください。 どうでしたか? 知らないと聞き取れなかったかもしれません。 代々木公園と言っていました。 今日は東京の渋谷にある代々木公園を紹介します。 まずは改札を出て、4番出口へ向かいます。 階段を登って、外に出ます。 右の方向に3分ぐらい歩いていくと公園の門があります。 ここから入りましょう。西門です。 地図を見てみましょう。 この現在地。現在はpresent。 地はplace、つまり今いる場所のことです。 広いですね。 代々木公園は東京で6番目に広い公園だそうです。 どこに行っていいかわからないので、とりあえずあちらに行きましょう。 走っている人がいますね。 と思ったら歩いていますね。 坂なんで疲れたんですかね。 立ち入りをご遠慮くださいと書いてあります。 入らないでくださいという意味ですね。 ちょっとこっちへ行ってみましょう。 落ち葉がたくさんありますね。 この日は11月26日でした。 なので公園を見るには11月中旬頃がいいかもしれないですね。 何かありますよ。 行ってみましょう